In [10]:
import pandas as pd
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import gc

gc.enable()
sns.set(font_scale=1.5)
pd.set_option('display.max_columns', 130)
pd.set_option('display.max_rows', 130)

In [11]:
# Carregando nomes e populações de municípios, estados e regiões (IBGE)
popregiao = pd.read_csv('data/dim_municipio/pop_regioes.csv')
popestado = pd.read_csv('data/dim_municipio/pop_estados.csv')
popmunicipio = pd.read_csv('data/dim_municipio/pop_municipios.csv')

# Adaptando tabs
popmunicipio[['cod_uf', 'cod_munic']] = popmunicipio[['cod_uf', 'cod_munic']].astype('str')
popmunicipio['cod_munic'] = popmunicipio['cod_munic'].str.zfill(5)
popmunicipio['uf_municipio'] = popmunicipio['uf'] + ' - ' + popmunicipio['municipio']
popmunicipio['cod_ufmunicipio'] = popmunicipio['cod_uf'] + popmunicipio['cod_munic']
popmunicipio['6cod_municipio'] = popmunicipio.apply(lambda x: x['cod_ufmunicipio'][:-1], axis=1)
popmunicipio.drop(columns=['cod_uf', 'cod_munic', 'cod_ufmunicipio', 'municipio', 'uf'], axis=1, inplace=True)
popmunicipio = popmunicipio[['6cod_municipio', 'uf_municipio', 'populacao']]

popmunicipio.head()

,6cod_municipio,uf_municipio,populacao
0,110001,RO - Alta Floresta D'Oeste,22945
1,110002,RO - Ariquemes,107863
2,110003,RO - Cabixi,5312
3,110004,RO - Cacoal,85359
4,110005,RO - Cerejeiras,16323


In [12]:
# Leitura de arquivos
usefulcols = ['AP_AUTORIZ', 'AP_TPAPAC', 'AP_MVM', 'AP_CNSPCN', 'AP_CMP', 'AP_CIDPRI', 'AQ_ESTADI', 'AQ_DTIDEN', 'AQ_DTINTR', 'AP_VL_AP', 'AP_OBITO', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_CEPPCN', 'AP_MUNPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_CODUNI', 'AP_PRIPAL', 'AP_UFMUN']

mama = pd.read_csv('data/quimio/mama_quimio_sia.csv', encoding='iso-8859-1', low_memory=False, usecols=usefulcols)
pulmao = pd.read_csv('data/quimio/pulmao_quimio_sia.csv', encoding='iso-8859-1', low_memory=False, usecols=usefulcols)
linfoma = pd.read_csv('data/quimio/linfomas_quimio_sia.csv', encoding='iso-8859-1', low_memory=False, usecols=usefulcols)
colonreto = pd.read_csv('data/quimio/colon_reto_quimio_sia.csv', encoding='iso-8859-1', low_memory=False, usecols=usefulcols)
prostata = pd.read_csv('data/quimio/prostata_quimio_sia.csv', encoding='iso-8859-1', low_memory=False, usecols=usefulcols)

# Unindo todos os datasets
df = pd.concat([mama, prostata, colonreto, linfoma, pulmao])

# Liberando 'base' da memória
del mama, prostata, colonreto, linfoma, pulmao
gc.collect()

print('Tamanho original do dataset geral: {}'.format(len(df)))

Tamanho original do dataset geral: 12205359


In [13]:
# Conversão das cols de tempo para data
df['AP_CMP'] = pd.to_datetime(df['AP_CMP'].astype(str), format='%Y%m', errors='coerce')#.apply(lambda x: x.strftime('%Y-%m'))
df['AQ_DTINTR'] = pd.to_datetime(df['AQ_DTINTR'].astype(str), format='%Y%m%d', errors='coerce')
df['AQ_DTIDEN'] = pd.to_datetime(df['AQ_DTIDEN'].astype(str), format='%Y%m%d', errors='coerce')

# Conversão de cols de município para string
df[['AP_UFMUN', 'AP_MUNPCN']] = df[['AP_UFMUN', 'AP_MUNPCN']].astype('str')

# Filtro de janela temporal
ano_inicio = 2014
df = df.loc[(df['AQ_DTINTR'].dt.year >= ano_inicio) & \
            (df['AQ_DTINTR'].dt.year >= ano_inicio) & (df['AQ_DTINTR'].dt.year <= 2019) & \
            (df['AQ_DTIDEN'].dt.year >= ano_inicio) & (df['AQ_DTIDEN'].dt.year <= 2019)]

In [14]:
# Adicionando nome do município do estabelecimento de saúde
df = pd.merge(df, popmunicipio, left_on='AP_UFMUN', right_on='6cod_municipio')
df.drop(columns=['6cod_municipio', 'populacao'], axis=1, inplace=True)
df.rename(columns={'uf_municipio': 'munic_estabel'}, inplace=True)

# Adicionando nome do município de residência do paciente
df = pd.merge(df, popmunicipio, left_on='AP_MUNPCN', right_on='6cod_municipio')
df.drop(columns=['6cod_municipio', 'populacao'], axis=1, inplace=True)
df.rename(columns={'uf_municipio': 'munic_resid'}, inplace=True)

In [15]:
# Ordenando linhas
df.sort_values(by=['AP_CNSPCN', 'AP_CMP'], ascending=[True, True], inplace=True)

# Demora para início do tratamento
df['demora_atd'] = (df['AQ_DTINTR'] - df['AQ_DTIDEN']).dt.days
demora = df.loc[df['demora_atd'] >= 0]
demora = demora[['AP_CNSPCN', 'demora_atd']].groupby('AP_CNSPCN').mean()['demora_atd']
df = pd.merge(df, demora.to_frame('demora_1_atd'), on='AP_CNSPCN')

# Classificação: diagnóstico precoce vs tardio
diagn = df.drop_duplicates(subset='AP_CNSPCN', keep='first')[['AP_CNSPCN', 'AQ_ESTADI']]
diagn.loc[diagn['AQ_ESTADI'].isin([0, 1, 2]), 'diagn_precoce'] = True
diagn.loc[diagn['AQ_ESTADI'].isin([3, 4]), 'diagn_precoce'] = False
df = pd.merge(df, diagn, on='AP_CNSPCN')
df.rename(columns={'AQ_ESTADI_y': 'AQ_ESTADI'}, inplace=True)

# Dropando colunas desnecessárias
df.drop(columns=['demora_atd', 'AQ_ESTADI_x'], axis=1, inplace=True)

# Reordenando colunas
usefulcols = ['AP_AUTORIZ', 'AP_TPAPAC', 'AP_MVM', 'AP_CNSPCN', 'AP_CMP', 'AP_CIDPRI', 'diagn_precoce', 'demora_1_atd', 'AQ_ESTADI', 'AQ_DTIDEN', 'AQ_DTINTR', 'AP_VL_AP', 'AP_OBITO', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_CEPPCN', 'AP_MUNPCN', 'munic_resid', 'AP_UFDIF', 'AP_MNDIF', 'AP_CODUNI', 'AP_PRIPAL', 'AP_UFMUN', 'munic_estabel']
df = df[usefulcols]

print('Tamanho do dataset após limpeza e organização gerais: {}'.format(len(df)))
df.head()

Tamanho do dataset após limpeza e organização gerais: 5754880


,AP_AUTORIZ,AP_TPAPAC,AP_MVM,AP_CNSPCN,AP_CMP,AP_CIDPRI,diagn_precoce,demora_1_atd,AQ_ESTADI,AQ_DTIDEN,AQ_DTINTR,AP_VL_AP,AP_OBITO,AP_NUIDADE,AP_SEXO,AP_RACACOR,AP_CEPPCN,AP_MUNPCN,munic_resid,AP_UFDIF,AP_MNDIF,AP_CODUNI,AP_PRIPAL,AP_UFMUN,munic_estabel
0,1215200137393,1,201507,|{{|{{{{{{,2015-07-01,"C33-C34- Câncer de traqueia, brônquio e pulmão",False,13.000000,3.0,2015-06-26,2015-07-09,1100.0,1,80,M,3,69900970,120040,AC - Rio Branco,0,1.0,2001586,304020222,120040,AC - Rio Branco
1,3514225060632,1,201406,|{{|{|~{{{~,2014-06-01,C18-C21- Câncer de cólon e reto,True,319.142857,2.0,2014-03-27,2014-06-25,2224.0,0,74,M,1,16290000,350770,SP - Braúna,0,1.0,2090236,304050024,350550,SP - Barretos
2,3514225060632,2,201407,|{{|{|~{{{~,2014-07-01,C18-C21- Câncer de cólon e reto,True,319.142857,2.0,2014-03-27,2014-06-25,2224.0,0,74,M,1,16290000,350770,SP - Braúna,0,1.0,2090236,304050024,350550,SP - Barretos
3,3514225060632,2,201408,|{{|{|~{{{~,2014-08-01,C18-C21- Câncer de cólon e reto,True,319.142857,2.0,2014-03-27,2014-06-25,2224.0,0,74,M,1,16290000,350770,SP - Braúna,0,1.0,2090236,304050024,350550,SP - Barretos
4,3514234374442,1,201409,|{{|{|~{{{~,2014-09-01,C18-C21- Câncer de cólon e reto,True,319.142857,2.0,2014-03-27,2014-06-25,2224.0,0,74,M,1,16290000,350770,SP - Braúna,0,1.0,2090236,304050024,350550,SP - Barretos


In [17]:
df.to_csv('sia_total.csv', index=False)